In [1]:
import cv2
from cv2.data import haarcascades
import os
from keras.models import load_model
import numpy as np
from pygame import mixer
import time

pygame 2.5.2 (SDL 2.28.3, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
mixer.init()
sound = mixer.Sound('assets/alert.wav')

# sound.play()
# time.sleep(0.2)
# sound.stop()
# time.sleep(0.5)
# sound.play()
# time.sleep(0.2)
# sound.stop()
# time.sleep(0.4)
# sound.play()
# time.sleep(0.2)
# sound.stop()
# time.sleep(0.3)
# sound.play()
# time.sleep(0.2)
# sound.stop()
# time.sleep(0.2)
# sound.play()
# time.sleep(0.2)
# sound.stop()
# time.sleep(0.1)
# sound.play()
# time.sleep(1)
# sound.stop()

In [3]:
face_cascade = cv2.CascadeClassifier(haarcascades + 'haarcascade_frontalface_default.xml')
left_eye_cascade = cv2.CascadeClassifier(haarcascades + 'haarcascade_lefteye_2splits.xml')
right_eye_cascade = cv2.CascadeClassifier(haarcascades + 'haarcascade_righteye_2splits.xml')

In [4]:
eye_model = load_model('models/CNN_eye_normal.keras')
yawn_model = load_model('models/yawn_detection_model.keras')

In [5]:
def get_leftmost_eye(eyes):
    leftmost = float('inf')
    leftmost_eye = None
    for i, eye in enumerate(eyes):
        if eye[0].any() < leftmost:
            leftmost = eye[0]
            leftmost_eye = eye
    return leftmost_eye

def get_rightmost_eye(eyes):
    rightmost = -1
    rightmost_eye = None
    for i, eye in enumerate(eyes):
        if eye[0].any() > rightmost:
            rightmost = eye[0]
            rightmost_eye = eye
    return rightmost_eye

In [20]:
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print('Error opening video capture')
    exit(1)

score = 0
score_threshold = 15

while True:
    ret, frame = cap.read()
    if not ret:
        print('Error reading video frame')
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        face = frame[y:y+h, x:x+w]
        face_gray = gray[y:y+h, x:x+w]

        eyes = []
        left_eye = left_eye_cascade.detectMultiScale(face_gray, 1.3, 5)
        for (ex, ey, ew, eh) in left_eye:
            eyes.append(face[ey:ey+eh, ex:ex+ew])
            # cv2.rectangle(face, (ex, ey), (ex+ew, ey+eh), (0, 255, 0), 2)
            # cv2.putText(face, 'Left Eye', (ex, ey), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        right_eye = right_eye_cascade.detectMultiScale(face_gray, 1.3, 5)
        for (ex, ey, ew, eh) in right_eye:
            eyes.append(face[ey:ey+eh, ex:ex+ew])
            # cv2.rectangle(face, (ex, ey), (ex+ew, ey+eh), (0, 255, 0), 2)
            # cv2.putText(face, 'Right Eye', (ex, ey), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        # cv2.putText(frame, 'Face', (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

        face_gray = cv2.resize(face_gray, (150, 150))
        face_gray = face_gray.reshape(1, 150, 150, 1)
        face_gray = face_gray / 255.0

        yawn_pred = yawn_model.predict(face_gray)
        if yawn_pred[0] > 0.5:
            # cv2.putText(frame, 'Yawn Detected', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            score += 1
            if score > score_threshold:
                score = score_threshold + 1
            # sound.play()
        else:
            score -= 1
            # sound.stop()

        leftmost_eye = get_leftmost_eye(left_eye)
        rightmost_eye = get_rightmost_eye(right_eye)
        for eye in [leftmost_eye, rightmost_eye]:
            if eye is None:
                continue
            print(eye)
            ex, ey, ew, eh = eye
            eye = face[ey:ey+eh, ex:ex+ew]
            # cv2.rectangle(face, (ex, ey), (ex+ew, ey+eh), (0, 255, 0), 2)
            # cv2.putText(face, 'Eye', (ex, ey), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            eye_gray = cv2.cvtColor(eye, cv2.COLOR_BGR2GRAY)
            eye_gray = cv2.resize(eye_gray, (250, 250))
            eye_gray = eye_gray.reshape(1, 250, 250, 1)
            eye_gray = eye_gray / 255.0
            
            eye_pred = eye_model.predict(eye_gray)
            if eye_pred[0] < 0.5:
                # cv2.putText(frame, 'Eye Closed', (x, y-30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                score += 1
                if score > score_threshold:
                    score = score_threshold + 1
                # sound.play()
            else:
                score -= 1
                # sound.stop()
        
        if score < 0:
            score = 0
        
        if score > score_threshold:
            cv2.putText(frame, 'Drowsiness Detected', (x, y-50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 1, cv2.LINE_AA)
            # check if sound is already playing
            if not mixer.get_busy():
                sound.play()
        else:
            sound.stop()

    cv2.imshow('frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()

        

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
[44 48 31 31]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
[39 43 31 31]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
[46 47 32 32]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
[43 48 26 26]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
[43 46 30 30]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
